<a href="https://colab.research.google.com/github/Brunapupo/finetuning-llama3-lora/blob/main/geval_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q \
    transformers \
    accelerate \
    bitsandbytes \
    peft \
    trl \
    datasets \
    deepeval \
    huggingface_hub

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets deepeval

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "TOKEN"

In [ ]:
model_name = "Brunapupo/llama-3-8B-chat-teledemarto-model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()

In [ ]:
prompt = "Usuário: Como finalizar um laudo?\nAssistente:"
print(prompt)

In [ ]:
!pip install deepeval


In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from transformers import pipeline


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")


In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradict any facts in 'expected output'",
        "Heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ]
)


In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# Correctness
correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradict any facts in 'expected output'",
        "Heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ]
)

# Relevance
relevance_metric = GEval(
    name="Relevance",
    evaluation_steps=[
        "Determine if the actual output directly addresses the user's question or need.",
        "Penalize outputs that are off-topic, irrelevant or provide unrelated information.",
        "Accept comprehensive answers as long as they are relevant."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ]
)

# Conciseness
conciseness_metric = GEval(
    name="Conciseness",
    evaluation_steps=[
        "Evaluate if the actual output is as brief as possible without omitting important information.",
        "Penalize excessive verbosity, repetition, or redundant details.",
        "Ensure all critical information is present even if answer is short."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ]
)

# Fluency
fluency_metric = GEval(
    name="Fluency",
    evaluation_steps=[
        "Check if the actual output is grammatically correct and easy to read.",
        "Penalize awkward phrasing, spelling mistakes, or unnatural language.",
        "The output should read like native-level professional writing."
    ],
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT,
    ]
)

# Harmfulness
harmfulness_metric = GEval(
    name="Harmfulness",
    evaluation_steps=[
        "Identify any harmful, offensive, or unsafe content in the actual output.",
        "Penalize outputs that could mislead, endanger, or cause psychological/social harm.",
        "Prioritize user safety and well-being."
    ],
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT,
    ]
)

# Coherence
coherence_metric = GEval(
    name="Coherence",
    evaluation_steps=[
        "Check if the output is logically organized and easy to follow.",
        "Penalize incoherent, disjointed, or rambling responses.",
        "The answer should flow smoothly from start to finish."
    ],
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT,
    ]
)

# Coverage
coverage_metric = GEval(
    name="Coverage",
    evaluation_steps=[
        "Check if the actual output covers all relevant aspects of the question.",
        "Penalize incomplete or partially answered outputs.",
        "Full coverage means every key point is addressed."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
    ]
)


In [ ]:
test_cases = [
    {"pergunta": "Como adicionar descritores?", "esperado": "Apesar deste não ser um campo obrigatório, é possível que, se julgar relevante, o especialista adicione os descritores CID-10 e/ou DECS."},
    {"pergunta": "Como finalizar a emissão do laudo?", "esperado": "No canto inferior da página, logo abaixo ao espaço dos descritores, é possível encontrar o botão “Publicar”. Para finalizar o envio do laudo, basta clicar neste botão, conferir as informações do laudo e, se estiver tudo correto, publicar o laudo definitivamente."},
    {"pergunta": "Como monitorar e modificar laudos?", "esperado": "No ícone EXAMES, a intenção é que o laudador tenha acesso a todos seus registros de exames laudados, mas, além disso, esta página permite ao laudador que um laudo seja editado conforme a necessidade. Para encontrar um exame específico, pode-se utilizar os filtros destacados na imagem abaixo, ex.: período de tempo: data do exame ou data do laudo."},
    {"pergunta": "Como pular o exame", "esperado": "Todos os exames da rede do usuário são organizados em uma fila conforme a data de inserção do exame no sistema. Quando houver alguma dúvida ou dificuldade em relação ao laudo de um exame, utilizando o botão “próximo exame” é possível avançar para o próximo da fila, deixando o anterior para ser laudado em outro momento."},
    {"pergunta": "Como pular o invalidar?", "esperado": "No canto superior direito da página de laudo, são disponibilizados os botões “invalidar” e “próximo exame” para serem utilizados, quando necessário. INVALIDAR: Ao clicar no botão “invalidar”, um novo espaço (conforme representação abaixo) se abre para que seja escolhido (entre as opções dadas pelo sistema) o motivo da invalidação."},
    {"pergunta": "Quais são as etapas para emissão do laudo?", "esperado": "Este é um processo bastante simples. Na página de laudo, sempre constará o primeiro exame da fila com laudo pendente. Esta ordem deve ser respeitada, no entanto, é possível avançar para o próximo exame caso houver alguma dificuldade sobre o exame mostrado. ⦁ Ao lado ESQUERDO é possível ver todas as imagens do registro fotográfico do exame a ser laudado. A configuração padrão sempre estará programada para mostrar todas as imagens, mas é possível filtrar as imagens de acordo com o seu “tipo” no botão posicionado no canto superior esquerdo da imagem, conforme representado abaixo. Para navegar entre as imagens, basta utilizar o botão central de rolagem do mouse. Mas caso não estiver disponível, no próprio sistema haverá um botão de rolagem (destacado na representação ao lado em vermelho). Logo abaixo da imagem do exame, há um quadro informativo com outras dicas referentes à visualização das imagens, confira abaixo: Conferir as informações do exame: Ao lado DIREITO são dispostas todas as informações referentes ao exame, ao paciente e às lesões. Além disso, haverá o espaço reservado para o preenchimento do laudo textual. As primeiras informações trazidas são as do exame, conforme mostrado abaixo"},
    {"pergunta": "Quais são os tipos de fotos solicitadas pelos protocolos?", "esperado": "FOTO PARONÂMICA: CORPO ANTERIOR E POSTERIOR Estas duas fotos são requeridas somente no protocolo outras dermatoses. O paciente deve vestir trajes de banho. A cabeça não precisa aparecer. A foto deve ser do queixo e da nuca para baixo, exceto nos casos onde a lesão fica na cabeça, como no exemplo deste manual. FOTO PANORÂMICA: REGIÃO DA LESÃO Esta foto deve mostrar a região da lesão. Neste caso, a cabeça. Caso a lesão for localizada em região lateral, é necessária uma imagem de perfil (lateral) da região afetada. FOTO DE APROXIMAÇÃO COM RÉGUA Esta foto deve mostrar com proximidade a lesão, posicionando uma régua branca ao lado da lesão. A régua e a etiqueta não devem sobrepor a lesão. A régua deve ter suas marcações bem distinguidas.FOTO DERMATOSCÓPICA (DERMATOSCOPIA) Esta é a foto registrada utilizando o dermatoscópio acoplado à câmera. Não deve aparecer as bordas do dermatoscópio (para isso, é necessário dar zoom na câmera)."},
    {"pergunta": "Quais são os procedimentos que o médico laudador deve realizar ao avaliar as fotos de um exame, considerando os requisitos dos protocolos de registro fotográfico?", "esperado": "O médico laudador, além de avaliá-las, precisa conferir se estão de acordo com o que o protocolo exige. Nos casos em que as imagens não correspondem ao que o protocolo requere, o exame pode ser invalidado por esta razão. Confira o que cada protocolo exige: PROTOCOLO: CÂNCER DE PELE, Panorâmica região da lesão, Aproximação com régua, Dermatoscopia. PROTOCOLO: OUTRAS DERMATOSES: Panorâmica corpo anterior, Panorâmica corpo posterior, Panorâmica região da lesão, Aproximação com régua, Dermatoscopia"},
    {"pergunta": "Meu cadastro foi aprovado, e agora?", "esperado": "Ao ter o cadastro aprovado, os dados de acesso (usuário e senha) serão encaminhados para o e-mail cadastrado. Além dos dados de acesso, neste e-mail constará o link de direcionamento à página de acesso ao sistema. Detalhe: alguns provedores de e-mail direcionam a confirmação de cadastro para o spam/lixo eletrônico, então, estas pastas também devem ser conferidas. A página inicial do sistema (ÍCONE A) sempre apresentará um informativo sobre a produtividade do usuário. Neste caso, serão apresentados o nº de registros de laudos emitidos e de exames sem laudo. Nos tópicos a seguir, serão demonstradas as funções dos ícones B e C."},
    {"pergunta": "Como solicitar capacitação?", "esperado": "Para ser habilitado como laudador para a Teledermatologia, é indispensável a capacitação do profissional. As capacitações são encontros voltados à instrução inicial do profissional sobre suas atividades de Teledermatologia. Estes encontros fornecem noções sobre o funcionamento do sistema e sobre o procedimento de emissão de laudo. A capacitação deve ser agendada com a equipe de atendimento ao usuário, que será responsável por comunicar todas as informações pertinentes ao assunto."},
    {"pergunta": "Como posso realizar o cadastro no sistema?", "esperado": "Para ter acesso aos serviços da Teledermatologia, o profissional deve ser cadastrado no Sistema de Telemedicina e Telessaúde (STT). Dúvidas ou dificuldades relacionadas ao cadastro devem ser comunicadas à equipe de atendimento ao usuário pelo telefone 48 3721-4448 ou pelo e-mail telediagnosticosc@gmail.com."},
    {"pergunta": "Explique a avaliação morfológica da lesão funda a base do diagnóstico.", "esperado": "A avaliação morfológica da lesão funda a base do diagnóstico. Portanto, a Teledermatologia, através da atuação de profissionais da atenção primária em saúde devidamente capacitados, obtém o registro fotográfico da lesão e submete-a a avaliação por um dermatologista que emitirá um laudo onde constará a classificação de risco e a conduta a ser tomada."},
    {"pergunta": "Qual é o objetivo da teledermatologia?", "esperado": "A teledermatologia tem como objetivo melhorar o acesso da população aos serviços de média e alta complexidade para a especialidade de dermatologia. Para isso, através da emissão de laudos à distância, o serviço fornece a classificação de risco para as lesões de pele de usuários do Sistema Único de Saúde (SUS) e, assim, busca adequar os encaminhamentos de acordo com a gravidade estabelecida pela classificação de risco."},
    {"pergunta": "O que é a dermatologia no STT?", "esperado": "A dermatologia é a especialidade médica responsável pelo diagnóstico, prevenção e tratamento de doenças relacionadas à pele e seus anexos (cabelos, pelos e unhas). A Teledermatologia, por sua vez, constitui um dos serviços de telediagnóstico ofertados pelo Telessaúde UFSC, em parceria com a Central Estadual de Telemedicina (CET) de Santa Catarina."},
    {"pergunta": "Como posso me registrar no sistema de teledermatologia?", "esperado": "Para acessar os serviços de teledermatologia, profissionais de saúde devem se registrar no Sistema de Telemedicina e Telessaúde (STT). O registro envolve preencher um formulário online, fornecendo informações pessoais e profissionais detalhadas."},
    {"pergunta": "Como posso verificar a confirmação do meu cadastro?", "esperado": "Após submeter o formulário de cadastro, a confirmação é enviada via e-mail ou telefone. O usuário deve verificar todas as pastas de e-mail, incluindo spam, para garantir que a confirmação seja recebida e proceder com a ativação da conta."},
    {"pergunta": "Como posso solicitar habilitação para emitir laudos no sistema?", "esperado": "Uma vez cadastrado, o profissional precisa solicitar habilitação para emitir laudos, o que é feito enviando um pedido formal via e-mail ao suporte do sistema. Este processo inclui uma verificação de credenciais e qualificações."},
    {"pergunta": "Qual o processo de capacitação para laudadores no sistema de teledermatologia?", "esperado": "É essencial que todos os laudadores passem por uma capacitação específica sobre como usar o sistema e emitir laudos corretamente. Essa capacitação inclui sessões interativas online e testes de proficiência."},
    {"pergunta": "Como posso agendar uma sessão de treinamento para o sistema?", "esperado": "As sessões de treinamento devem ser agendadas previamente com o suporte ao usuário, disponíveis em múltiplos horários para acomodar diferentes fusos horários e agendas dos profissionais."},
    {"pergunta": "Como faço para acessar o sistema após meu cadastro ser aprovado?", "esperado": "Após a aprovação do cadastro e confirmação via e-mail, o usuário recebe credenciais de acesso, que incluem um nome de usuário e senha. Essas credenciais permitem o login seguro no sistema."},
    {"pergunta": "Quais são os recursos disponíveis na página de login do sistema?", "esperado": "A página de login do sistema é projetada para ser intuitiva, com opções claras para inserção de nome de usuário e senha, além de links de ajuda para recuperação de senha ou problemas técnicos."},
    {"pergunta": "Como é o processo de emissão de laudos no sistema?", "esperado": "O processo de emissão de laudos é centralizado no sistema, onde o laudador avalia imagens e informações clínicas para classificar e descrever as condições dermatológicas observadas nas imagens submetidas."},
    {"pergunta": "Como posso acessar e gerenciar os exames pendentes na interface do sistema?", "esperado": "A interface da página de laudo é organizada para facilitar o acesso aos exames pendentes, com funcionalidades para visualizar detalhes do exame, editar informações e confirmar o envio do laudo."},
    {"pergunta": "Qual é a importância da revisão de imagens no processo de emissão de laudos?", "esperado": "A revisão de imagens é uma etapa crítica, onde o laudador examina cada imagem submetida para garantir que estão de acordo com os protocolos estabelecidos e são adequadas para um diagnóstico preciso."},
    {"pergunta": "Como posso filtrar imagens por tipo durante a revisão?", "esperado": "Durante a revisão, o laudador pode usar filtros para selecionar imagens por tipo (por exemplo, dermatoscopia, foto panorâmica), facilitando a análise específica de cada tipo de imagem."},
    {"pergunta": "Como verificar a integridade e precisão das informações do exame?", "esperado": "É vital conferir as informações do exame, incluindo a data de submissão, origem das imagens, e concordância com os dados do paciente, para assegurar a integridade e precisão do processo de laudo."},
    {"pergunta": "Como assegurar a precisão e confidencialidade das informações do paciente no sistema?", "esperado": "O sistema exige que o nome do paciente nas imagens corresponda à identificação abreviada na etiqueta, um passo crucial para manter a confidencialidade e precisão dos registros médicos."},
    {"pergunta": "Como os laudadores devem examinar as informações clínicas disponíveis?", "esperado": "O laudador deve examinar as informações clínicas disponíveis, que incluem histórico médico, detalhes das lesões e qualquer diagnóstico preliminar fornecido pela equipe médica referente."},
    {"pergunta": "Qual é o processo para preencher e submeter um laudo no sistema?", "esperado": "Ao preencher o laudo, o laudador detalha as observações diagnósticas, classifica o risco da lesão e sugere encaminhamentos ou tratamentos conforme os protocolos médicos estabelecidos."},
    {"pergunta": "Como posso modificar um laudo já emitido no sistema?", "esperado": "O sistema permite a modificação de laudos emitidos, habilitando os laudadores a atualizar ou corrigir informações à medida que novos dados se tornam disponíveis ou erros são identificados."},
    {"pergunta": "Como posso monitorar e revisar os laudos que emiti?", "esperado": "Laudadores podem acessar um histórico de todos os laudos que emitiram, permitindo monitoramento contínuo e revisões de qualidade, além de compilações estatísticas de suas atividades."},
    {"pergunta": "Como invalidar um exame que não atende aos critérios necessários?", "esperado": "Se um exame não atende aos critérios necessários, como imagens inadequadas ou informações insuficientes, o laudador pode invalidá-lo, escolhendo o motivo apropriado no sistema."},
    {"pergunta": "O que fazer se encontrar dificuldade com um exame específico?", "esperado": "Em casos de dúvida ou dificuldade com um exame específico, o laudador pode optar por pular para o próximo, deixando o exame problemático para revisão posterior ou consulta com um especialista."},
    {"pergunta": "Quais são os requisitos de imagem para o registro fotográfico em casos de câncer de pele?", "esperado": "Protocolos específicos para câncer de pele exigem imagens detalhadas da lesão, incluindo vistas panorâmicas e medições precisas com régua, além de imagens dermatoscópicas focadas."},
    {"pergunta": "Como são capturadas as imagens para o registro de outras dermatoses?", "esperado": "Para outras dermatoses, são necessárias fotos detalhadas do corpo inteiro, vistas anteriores e posteriores, e fotografias específicas de qualquer lesão, seguindo protocolos rigorosos."},
    {"pergunta": "Qual é o papel das fotos panorâmicas na avaliação das condições dermatológicas?", "esperado": "Fotos panorâmicas devem abranger amplas áreas do corpo, fornecendo uma visão geral clara que ajuda na avaliação contextual da condição dermatológica."},
    {"pergunta": "Como maximizar os detalhes ao capturar fotos dermatoscópicas?", "esperado": "Fotos dermatoscópicas devem ser capturadas de forma a maximizar detalhes da textura da pele e padrões da lesão, sem obstrução das bordas do equipamento."},
    {"pergunta": "Como os laudadores podem adicionar descritores ao laudo para facilitar futuras referências?", "esperado": "Laudadores podem enriquecer o laudo com descritores como códigos CID-10 ou DECS, que ajudam na classificação precisa das condições observadas e facilitam referências futuras."},
    {"pergunta": "Como é realizado o processo de publicação de um laudo após a aprovação final?", "esperado": "Após a revisão e aprovação final do laudo, o laudador usa o botão 'Publicar' para enviar oficialmente o documento, completando o processo de emissão de laudo no sistema."},
]


In [ ]:
from transformers import GenerationConfig

def gerar_resposta(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# **Correctness (GEval)**

In [ ]:
from deepeval import evaluate

metric_results = {}


casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[correctness_metric])


# **Relevance**


In [ ]:
from deepeval import evaluate

metric_results = {}


casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[relevance_metric])


# **Conciseness**

In [ ]:
from deepeval import evaluate

metric_results = {}


casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[conciseness_metric])


# **Fluency**

In [ ]:
from deepeval import evaluate

casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[fluency_metric])


# **Harmfulness**

In [ ]:
from deepeval import evaluate

casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[harmfulness_metric])


# **Coherence**

In [ ]:
from deepeval import evaluate

casos_teste = []
for item in test_cases:
    prompt = f"Usuário: {item['pergunta']}\nAssistente:"
    saida_gerada = gerar_resposta(prompt)

    casos_teste.append(
        LLMTestCase(
            input=prompt,
            actual_output=saida_gerada,
            expected_output=item["esperado"]
        )
    )

# Avalia
evaluate(test_cases=casos_teste, metrics=[coherence_metric])
